In [ ]:
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.manifold import TSNE
from scipy.stats import ttest_ind
import xgboost as xgb
import pyarrow.parquet as pq
import anndata as ad
from cellrank.kernels import PseudotimeKernel
import ehrapy as ep
import cellrank as cr
from matplotlib.colors import TwoSlopeNorm, BoundaryNorm
import matplotlib.colors as mcolors
import warnings
import faiss
from joblib import Parallel, delayed
from sklearn.manifold import TSNE
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
import scanpy as sc
warnings.filterwarnings("ignore")

# Proxy settings
os.environ['http_proxy'] = 'http://127.0.0.1:6667'
os.environ['https_proxy'] = 'http://127.0.0.1:6667'
os.environ['all_proxy'] = 'socks5://127.0.0.1:6666'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [ ]:
protein_age_sex = pd.read_parquet('../adata_file/protein_age_sex_imputed.parquet')
protein_list = protein_age_sex.columns[3:-1]
proage20 = ["ACRV1", "AGRP", "CDCP1", "COL6A3", "CXCL17", "EDA2R", "ELN", "ENG", "FSHB", "GDF15", "GFAP", "IGDCC4", "KLK3", "KLK7", "LECT2", "LTBP2", "NEFL", "PODXL2", "PTPRR", "SCARF2"]
proage20 = [item.lower() for item in proage20]
effective_feature_age = pd.read_parquet('../adata_file/effective_feature_age.parquet')['effective_feature_age']
effective_feature_sex = pd.read_parquet('../adata_file/effective_feature_sex.parquet')['effective_feature_sex']
effective_feature_age_female = pd.read_parquet('../adata_file/effective_feature_age_female.parquet')['effective_feature_age_female']
effective_feature_age_male = pd.read_parquet('../adata_file/effective_feature_age_male.parquet')['effective_feature_age_male']

In [ ]:
protein_age_sex = pd.read_parquet('protein_age_sex_tsne.parquet')
protein_age_sex_filled = protein_age_sex[(protein_age_sex['p31']==1)]

In [ ]:
adata_male_effective_feature_age_male = ad.AnnData(X=protein_age_sex_filled[protein_age_sex_filled['p31']==1][proage20].values,var = pd.DataFrame(index=protein_age_sex_filled[protein_age_sex_filled['p31']==1][proage20].columns))
adata_male_effective_feature_age_male.obs['true_age'] = list(protein_age_sex_filled[protein_age_sex_filled['p31']==1]['true_age'].dropna())
adata_male_effective_feature_age_male.obs['true_agepredmale'] = list(protein_age_sex_filled[protein_age_sex_filled['p31']==1]['true_agepredmale'].dropna())
adata_male_effective_feature_age_male.obs['true_agepred'] = list(protein_age_sex_filled[protein_age_sex_filled['p31']==1]['true_agepred'].dropna())
adata_male_effective_feature_age_male.obs['eid'] = list(protein_age_sex_filled[protein_age_sex_filled['p31']==1]['eid'].dropna())
ep.pp.neighbors(adata_male_effective_feature_age_male, n_pcs=10)
ep.tl.tsne(adata_male_effective_feature_age_male,random_state=42,n_jobs = -1)
ep.tl.leiden(adata_male_effective_feature_age_male, resolution=0.3, key_added="leiden_0_3")

In [ ]:
ep.tl.umap(adata_male_effective_feature_age_male,random_state=42)

In [ ]:
adata_male_effective_feature_age_male.write('adata_male_effective_feature_age_male.h5ad')

In [ ]:
adata_male_effective_feature_age_male = ad.read('adata_male_effective_feature_age_male.h5ad')
adata_male_effective_feature_age_male.obs['x_zuobiao'] = adata_male_effective_feature_age_male.obsm['X_tsne'][:,0]
adata_male_effective_feature_age_male.obs['y_zuobiao'] = adata_male_effective_feature_age_male.obsm['X_tsne'][:,1]
adata_male_effective_feature_age_male.obs['x_umap'] = adata_male_effective_feature_age_male.obsm['X_umap'][:,0]
adata_male_effective_feature_age_male.obs['y_umap'] = adata_male_effective_feature_age_male.obsm['X_umap'][:,1]

### root sample

In [ ]:
adata_male_effective_feature_age_male.obs[adata_male_effective_feature_age_male.obs['true_age']<40.3]

### diffmap caculation

In [ ]:
# adata_male_effective_feature_age_male.uns["iroot"] = 1024
adata_male_effective_feature_age_male.uns["iroot"] = 1349
ep.tl.diffmap(adata_male_effective_feature_age_male)
ep.tl.dpt(adata_male_effective_feature_age_male)
# 'dpt_pseudotime' 'X_diffmap'

### transition—matrix

In [ ]:
pk_adata_both_sex_women= PseudotimeKernel(adata_male_effective_feature_age_male, time_key="dpt_pseudotime")
pk_adata_both_sex_women.compute_transition_matrix()

In [ ]:
adata_male_effective_feature_age_male = adata_male_effective_feature_age_male[(adata_male_effective_feature_age_male.obs['true_age']>=40)&(adata_male_effective_feature_age_male.obs['true_age']<=70)]
adata_male_effective_feature_age_male = adata_male_effective_feature_age_male[adata_male_effective_feature_age_male.obs['dpt_pseudotime']<0.4]
adata_male_effective_feature_age_male.obs['dpt_pseudotime'] = adata_male_effective_feature_age_male.obs['dpt_pseudotime']*(1/0.4)

### age mapping

In [ ]:
sorted_indices = np.argsort(list(adata_male_effective_feature_age_male.obs['dpt_pseudotime']))
ranked_positions = np.argsort(sorted_indices)
sorted_list = sorted(adata_male_effective_feature_age_male.obs['true_age'])
adata_male_effective_feature_age_male.obs['dpt_pseudotime_map'] = [sorted_list[i] for i in ranked_positions]

### compare diffusion supervised

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

# Set font style for scientific plotting
mpl.rcParams['font.family'] = 'DejaVu Sans'  # Or choose another font suitable for scientific plots
mpl.rcParams['font.size'] = 10  # Adjust font size as needed for clarity in scientific papers

# Create figure with subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5), dpi=300)  # 8x4 inches, DPI 300 for high quality

# Plot the first scatter plot
sns.set_palette("muted")
scatter1 = sns.scatterplot(
    data=adata_male_effective_feature_age_male.obs, 
    x='x_zuobiao', 
    y='y_zuobiao', 
    hue='dpt_pseudotime_map', 
    palette='viridis', 
    s=2, 
    edgecolor=None,
    legend=False,
    ax=axes[0]
)
axes[0].set_xlabel('TSNE-1')
axes[0].set_ylabel('TSNE-2')

# Add colorbar for the first plot
norm1 = plt.Normalize(adata_male_effective_feature_age_male.obs['dpt_pseudotime_map'].min(), adata_male_effective_feature_age_male.obs['dpt_pseudotime_map'].max())
sm1 = plt.cm.ScalarMappable(cmap="viridis", norm=norm1)
sm1.set_array([])
cbar1 = fig.colorbar(sm1, ax=axes[0], pad=0.02, aspect=20)
cbar1.set_label('Predicted BA (Ours)', fontsize=8)  # Set smaller label font for scientific presentation

# Plot the second scatter plot
scatter2 = sns.scatterplot(
    data=adata_male_effective_feature_age_male.obs, 
    x='x_zuobiao', 
    y='y_zuobiao', 
    hue='true_age', 
    palette='viridis', 
    s=2, 
    edgecolor=None,
    legend=False,
    ax=axes[1]
)
axes[1].set_xlabel('TSNE-1')
axes[1].set_ylabel('TSNE-2')

# Add colorbar for the second plot
norm2 = plt.Normalize(adata_male_effective_feature_age_male.obs['true_age'].min(), adata_male_effective_feature_age_male.obs['true_age'].max())
sm2 = plt.cm.ScalarMappable(cmap="viridis", norm=norm2)
sm2.set_array([])
cbar2 = fig.colorbar(sm2, ax=axes[1], pad=0.02, aspect=20)
cbar2.set_label('CA', fontsize=8)

scatter3 = sns.scatterplot(
    data=adata_male_effective_feature_age_male.obs, 
    x='x_zuobiao', 
    y='y_zuobiao', 
    hue='true_agepredmale', 
    palette='viridis', 
    s=2, 
    edgecolor=None,
    legend=False,
    ax=axes[2]
)
axes[2].set_xlabel('TSNE-1')
axes[2].set_ylabel('TSNE-2')

# Add colorbar for the first plot
norm3 = plt.Normalize(adata_male_effective_feature_age_male.obs['true_agepredmale'].min(), adata_male_effective_feature_age_male.obs['true_agepredmale'].max())
sm3 = plt.cm.ScalarMappable(cmap="viridis", norm=norm1)
sm3.set_array([])
cbar3 = fig.colorbar(sm1, ax=axes[2], pad=0.02, aspect=20)
cbar3.set_label('Predicted BA (Supervised)', fontsize=8)  # Set smaller label font for scientific presentation

# Adjust layout and save
plt.tight_layout()
plt.savefig("./figs-method/tsen_male_plots.png", dpi=600, bbox_inches='tight')  # Save as high-quality PNG
plt.show()

### compare mapping before vs after

In [ ]:
adata_male_effective_feature_age_male = adata_male_effective_feature_age_male[adata_male_effective_feature_age_male.obs['dpt_pseudotime']<0.6]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

# Set font style for scientific plotting
mpl.rcParams['font.family'] = 'DejaVu Sans'  # Or choose another font suitable for scientific plots
mpl.rcParams['font.size'] = 10  # Adjust font size as needed for clarity in scientific papers

# Create figure with subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 5), dpi=300)  # 8x4 inches, DPI 300 for high quality

# Plot the first scatter plot
sns.set_palette("muted")
scatter1 = sns.scatterplot(
    data=adata_male_effective_feature_age_male.obs, 
    x='x_zuobiao', 
    y='y_zuobiao', 
    hue='dpt_pseudotime', 
    palette='viridis', 
    s=2, 
    edgecolor=None,
    legend=False,
    ax=axes[0]
)
axes[0].set_xlabel('TSNE-1')
axes[0].set_ylabel('TSNE-2')

# Add colorbar for the first plot
norm1 = plt.Normalize(adata_male_effective_feature_age_male.obs['dpt_pseudotime'].min(), adata_male_effective_feature_age_male.obs['dpt_pseudotime'].max())
sm1 = plt.cm.ScalarMappable(cmap="viridis", norm=norm1)
sm1.set_array([])
cbar1 = fig.colorbar(sm1, ax=axes[0], pad=0.02, aspect=20)
cbar1.set_label('Pseudotime', fontsize=8)  # Set smaller label font for scientific presentation

# Plot the second scatter plot
scatter2 = sns.scatterplot(
    data=adata_male_effective_feature_age_male.obs, 
    x='x_zuobiao', 
    y='y_zuobiao', 
    hue='dpt_pseudotime_map', 
    palette='viridis', 
    s=2, 
    edgecolor=None,
    legend=False,
    ax=axes[1]
)
axes[1].set_xlabel('TSNE-1')
axes[1].set_ylabel('TSNE-2')

# Add colorbar for the second plot
norm2 = plt.Normalize(adata_male_effective_feature_age_male.obs['dpt_pseudotime_map'].min(), adata_male_effective_feature_age_male.obs['dpt_pseudotime_map'].max())
sm2 = plt.cm.ScalarMappable(cmap="viridis", norm=norm2)
sm2.set_array([])
cbar2 = fig.colorbar(sm2, ax=axes[1], pad=0.02, aspect=20)
cbar2.set_label('Predicted BA (Ours)', fontsize=8)

# Adjust layout and save
plt.tight_layout()
plt.savefig("./figs-method/tsne_male_plots_dpt_map_no_map<0.4.png", dpi=600, bbox_inches='tight')  # Save as high-quality PNG
plt.show()

# pk_adata_both_sex_male= PseudotimeKernel(adata_male_effective_feature_age_male, time_key="dpt_pseudotime")
# pk_adata_both_sex_male.compute_transition_matrix()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde


# 设置字体（如需科研绘图效果）
plt.rcParams['font.family'] = 'DejaVu Sans'  # 替换为其他字体如 'Times New Roman' 根据需要
plt.rcParams['font.size'] = 12

# 创建图形和子图
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
columns = ['true_age','dpt_pseudotime','dpt_pseudotime_map']
columnss = ['CA','Pseudotime','predicted BA after mapping']

# 绘制每列的 KDE
for i, col in enumerate(columns):
    sns.kdeplot(adata_male_effective_feature_age_male.obs[col], ax=axs[i], fill=True)
    axs[i].set_title(f'Distribution of {columnss[i]}')
    axs[i].set_xlabel(columnss[i])
    axs[i].set_ylabel('Density')
plt.tight_layout()
plt.show()
# plt.savefig("./figs-method/male-age-dist<0.4.png", dpi=600, bbox_inches='tight')  # Save as high-quality PNG

# data1 = adata_male_effective_feature_age_male.obs['dpt_pseudotime']*30+40
# data2 = adata_male_effective_feature_age_male.obs['dpt_pseudotime_map']
# # 使用gaussian_kde拟合这两个分布
# kde1 = gaussian_kde(data1)
# kde2 = gaussian_kde(data2)
# x = np.linspace(min(data1.min(), data2.min()), max(data1.max(), data2.max()), 1000)
# kde1_values = kde1.evaluate(x)
# kde2_values = kde2.evaluate(x)

# # 将第二个分布除以第一个分布
# kde_ratio = kde2_values / kde1_values
# sns.kdeplot(kde_ratio, ax=axs[3], fill=True)


In [ ]:
# adata_male_effective_feature_age_male = adata_male_effective_feature_age_male[(adata_male_effective_feature_age_male.obs['true_age']>=40)&(adata_male_effective_feature_age_male.obs['true_age']<=70)]

In [ ]:
adata_male_effective_feature_age_male.obs['true_age_int'] = adata_male_effective_feature_age_male.obs['true_age']//0.1
adata_male_effective_feature_age_male.obs['true_age_int'] = adata_male_effective_feature_age_male.obs['true_age_int'].astype(int)
# adata_male_effective_feature_age_male.obs['X_tsne_1'] = adata_male_effective_feature_age_male.obsm['X_tsne'][:,0]
# adata_male_effective_feature_age_male.obs['X_tsne_2'] = adata_male_effective_feature_age_male.obsm['X_tsne'][:,1]
def top_20_percent(group):
    return group.nlargest(int(len(group) * 0.2), 'dpt_pseudotime_map')
# 应用这个函数到每个组
top_20_percent_rows_dpt = adata_male_effective_feature_age_male.obs.groupby('true_age_int').apply(top_20_percent)
# adata_female_effective_feature_age_female.obs['top20_dpt'] = 
def top_20_percent(group):
    return group.nlargest(int(len(group) * 0.2), 'true_agepredmale')
top_20_percent_rows_supervised = adata_male_effective_feature_age_male.obs.groupby('true_age_int').apply(top_20_percent)

### fit line predict vs true

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib as mpl

# Set scientific-style font and DPI for saving
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['font.size'] = 10

# Sample data (replace 'df_filled' with your actual dataframe and columns)
# df_filled = your DataFrame
x1 = adata_male_effective_feature_age_male.obs['true_age']
y1 = adata_male_effective_feature_age_male.obs['dpt_pseudotime_map']

x11 = top_20_percent_rows_dpt['true_age']
y11 = top_20_percent_rows_dpt['dpt_pseudotime_map']

x2 = adata_male_effective_feature_age_male.obs['true_agepredmale']
y2 = adata_male_effective_feature_age_male.obs['dpt_pseudotime_map']
x3 = adata_male_effective_feature_age_male.obs['true_age']
y3 = adata_male_effective_feature_age_male.obs['true_agepredmale']

x33 = top_20_percent_rows_supervised['true_age']
y33 = top_20_percent_rows_supervised['true_agepredmale']
# Calculate metrics for the first plot
pcc1, _ = pearsonr(x1, y1)
r_squared1 = pcc1 ** 2
rmse1 = np.sqrt(mean_squared_error(x1, y1))
mae1 = mean_absolute_error(x1, y1)

# Calculate metrics for the second plot
pcc2, _ = pearsonr(x2, y2)
r_squared2 = pcc2 ** 2
rmse2 = np.sqrt(mean_squared_error(x2, y2))
mae2 = mean_absolute_error(x2, y2)

# Calculate metrics for the second plot
pcc3, _ = pearsonr(x3, y3)
r_squared3 = pcc3 ** 2
rmse3 = np.sqrt(mean_squared_error(x3, y3))
mae3 = mean_absolute_error(x3, y3)

# Set up the subplot structure
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True, dpi=300)
sns.set_palette("muted")

# Plot the first subplot
sns.regplot(x=x1, y=y1, scatter_kws={'s': 0.5, 'alpha': 0.5, 'color': 'darkblue'}, line_kws={'color': 'red'}, ax=axes[0])
# sns.scatterplot(x='true_age', y='dpt_pseudotime_map', data=top_20_percent_rows_dpt, s = 5,color='red', alpha=0.5, ax=axes[0])

axes[0].set_xlabel("CA")
axes[0].set_ylabel("Predicted BA (Ours)")
axes[0].set_xlim(min(x1) - 5, max(x1) + 5)
axes[0].set_ylim(min(y1) - 5, max(y1) + 5)
axes[0].text(min(x1)-3, max(y1)+3, f"$r = {pcc1:.2f}$\n$R^2 = {r_squared1:.2f}$\nRMSE = {rmse1:.2f}\nMAE = {mae1:.2f}",
             ha='left', va='top', fontsize=10, style='italic')

# Plot the second subplot
sns.regplot(x=x2, y=y2, scatter_kws={'s': 0.5, 'alpha': 0.5, 'color': 'darkblue'}, line_kws={'color': 'red'}, ax=axes[1])
axes[1].set_xlabel("Predicted BA (Supervised)")
axes[1].set_ylabel("Predicted BA (Ours)")
axes[1].set_xlim(min(x2) - 5, max(x2) + 5)
axes[1].text(min(x2)-3, max(y2)+3, f"$r = {pcc2:.2f}$\n$R^2 = {r_squared2:.2f}$\nRMSE = {rmse2:.2f}\nMAE = {mae2:.2f}",
             ha='left', va='top', fontsize=10, style='italic')


# Plot the second subplot
sns.regplot(x=x3, y=y3, scatter_kws={'s': 0.5, 'alpha': 0.5, 'color': 'darkblue'}, line_kws={'color': 'red'}, ax=axes[2])
# sns.scatterplot(x='true_age', y='true_agepredmale', data=top_20_percent_rows_supervised, s = 5,color='red', alpha=0.5, ax=axes[2])
axes[2].set_xlabel("CA")
axes[2].set_ylabel("Predicted Age (Supervised)")
axes[2].set_xlim(min(x2) - 5, max(x2) + 5)
axes[2].text(min(x3)-3, max(y3)+3, f"$r = {pcc3:.2f}$\n$R^2 = {r_squared3:.2f}$\nRMSE = {rmse3:.2f}\nMAE = {mae3:.2f}",
             ha='left', va='top', fontsize=10, style='italic')

# Adjust layout and save the plot
plt.tight_layout()
plt.savefig("./figs-method/Pseudotime-Predicted Age Male_nored.png", dpi=600)  # Save with specified DPI
plt.show()

In [ ]:
adata_male_effective_feature_age_male.obs['dpt_pseudotime_map_zscore'] = adata_male_effective_feature_age_male.obs.groupby('true_age_int')['dpt_pseudotime_map'].transform(lambda x: (x - x.mean()) / x.std() + x.mean())
adata_male_effective_feature_age_male.obs['true_agepredmale_zscore'] = adata_male_effective_feature_age_male.obs.groupby('true_age_int')['true_agepredmale'].transform(lambda x: (x - x.mean()) / x.std() + x.mean())

def top_20_percent(group):
    return group.nlargest(int(len(group) * 0.2), 'dpt_pseudotime_map_zscore')
# 应用这个函数到每个组
top_20_percent_rows_dpt = adata_male_effective_feature_age_male.obs.groupby('true_age_int').apply(top_20_percent)
# adata_female_effective_feature_age_female.obs['top20_dpt'] = 
def top_20_percent(group):
    return group.nlargest(int(len(group) * 0.2), 'true_agepredmale_zscore')
top_20_percent_rows_supervised = adata_male_effective_feature_age_male.obs.groupby('true_age_int').apply(top_20_percent)


import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib as mpl

# Set scientific-style font and DPI for saving
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['font.size'] = 10

# Sample data (replace 'df_filled' with your actual dataframe and columns)
# df_filled = your DataFrame
x1 = adata_male_effective_feature_age_male.obs[~adata_male_effective_feature_age_male.obs['dpt_pseudotime_map_zscore'].isna()]['true_age']
y1 = adata_male_effective_feature_age_male.obs[~adata_male_effective_feature_age_male.obs['dpt_pseudotime_map_zscore'].isna()]['dpt_pseudotime_map_zscore']
x2 = adata_male_effective_feature_age_male.obs[~adata_male_effective_feature_age_male.obs['dpt_pseudotime_map_zscore'].isna()]['true_agepredmale_zscore']
y2 = adata_male_effective_feature_age_male.obs[~adata_male_effective_feature_age_male.obs['dpt_pseudotime_map_zscore'].isna()]['dpt_pseudotime_map_zscore']
x3 = adata_male_effective_feature_age_male.obs[~adata_male_effective_feature_age_male.obs['dpt_pseudotime_map_zscore'].isna()]['true_age']
y3 = adata_male_effective_feature_age_male.obs[~adata_male_effective_feature_age_male.obs['dpt_pseudotime_map_zscore'].isna()]['true_agepredmale_zscore']


# Calculate metrics for the first plot
pcc1, _ = pearsonr(x1, y1)
r_squared1 = pcc1 ** 2
rmse1 = np.sqrt(mean_squared_error(x1, y1))
mae1 = mean_absolute_error(x1, y1)

# Calculate metrics for the second plot
pcc2, _ = pearsonr(x2, y2)
r_squared2 = pcc2 ** 2
rmse2 = np.sqrt(mean_squared_error(x2, y2))
mae2 = mean_absolute_error(x2, y2)

# Calculate metrics for the second plot
pcc3, _ = pearsonr(x3, y3)
r_squared3 = pcc3 ** 2
rmse3 = np.sqrt(mean_squared_error(x3, y3))
mae3 = mean_absolute_error(x3, y3)

# Set up the subplot structure
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True, dpi=300)
sns.set_palette("muted")

# Plot the first subplot
sns.regplot(x=x1, y=y1, scatter_kws={'s': 0.5, 'alpha': 0.5, 'color': 'darkblue'}, line_kws={'color': 'red'}, ax=axes[0])
# sns.scatterplot(x='true_age', y='dpt_pseudotime_map_zscore', data=top_20_percent_rows_dpt, s = 5,color='red', alpha=0.5, ax=axes[0])

axes[0].set_xlabel("CA")
axes[0].set_ylabel("Predicted BA (Ours)")
axes[0].set_xlim(min(x1) - 5, max(x1) + 5)
axes[0].set_ylim(min(y1) - 5, max(y1) + 5)
axes[0].text(min(x1)-3, max(y1)+3, f"$r = {pcc1:.2f}$\n$R^2 = {r_squared1:.2f}$\nRMSE = {rmse1:.2f}\nMAE = {mae1:.2f}",
             ha='left', va='top', fontsize=10, style='italic')

# Plot the second subplot
sns.regplot(x=x2, y=y2, scatter_kws={'s': 0.5, 'alpha': 0.5, 'color': 'darkblue'}, line_kws={'color': 'red'}, ax=axes[1])
axes[1].set_xlabel("Predicted BA (Supervised)")
axes[1].set_ylabel("Predicted BA (Ours)")
axes[1].set_xlim(min(x2) - 5, max(x2) + 5)
axes[1].text(min(x2)-3, max(y2)+3, f"$r = {pcc2:.2f}$\n$R^2 = {r_squared2:.2f}$\nRMSE = {rmse2:.2f}\nMAE = {mae2:.2f}",
             ha='left', va='top', fontsize=10, style='italic')


# Plot the second subplot
sns.regplot(x=x3, y=y3, scatter_kws={'s': 0.5, 'alpha': 0.5, 'color': 'darkblue'}, line_kws={'color': 'red'}, ax=axes[2])
# sns.scatterplot(x='true_age', y='true_agepredmale_zscore', data=top_20_percent_rows_supervised, s = 5,color='red', alpha=0.5, ax=axes[2])
axes[2].set_xlabel("CA")
axes[2].set_ylabel("Predicted Age (Supervised)")
axes[2].set_xlim(min(x3) - 5, max(x3) + 5)
axes[2].text(min(x3)-3, max(y3)+3, f"$r = {pcc3:.2f}$\n$R^2 = {r_squared3:.2f}$\nRMSE = {rmse3:.2f}\nMAE = {mae3:.2f}",
             ha='left', va='top', fontsize=10, style='italic')

# Adjust layout and save the plot
plt.tight_layout()
plt.savefig("./figs-method/Pseudotime-Predicted Age Male_zscore_nored.png", dpi=600)  # Save with specified DPI
plt.show()

In [ ]:
from sklearn.cluster import KMeans
# 执行 K-means 聚类
# 假设 adata 是你的 AnnData 对象
# 提取数据矩阵
X = adata_male_effective_feature_age_male.X
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
adata_male_effective_feature_age_male.obs['kmeans'] = kmeans.labels_
adata_male_effective_feature_age_male.obs['kmeans'] = adata_male_effective_feature_age_male.obs['kmeans'].map({1:0,0:1,2:2,3:3})

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# 假设您已经有一个AnnData对象 adata_female_effective_feature_sex_female_subsample
# 从 AnnData 对象中提取数据
df = adata_male_effective_feature_age_male.obs[['true_age', 'kmeans']]

# 使用科研绘图的样式
sns.set(style="whitegrid", font="DejaVu Sans", font_scale=1.2)

# 绘制 violin plot
plt.figure(figsize=(10, 5))
sns.violinplot(data=df, x="kmeans", y="true_age", palette="muted")

# 设置标签和样式
plt.xlabel("Cluster - male")
plt.ylabel(None)
plt.xticks(rotation=0)

# 保存高分辨率图像
# plt.savefig("./figs-method/violin_age_feat_male_true_age.png", dpi=600, bbox_inches="tight")
plt.show()


In [ ]:
# ep.tl.leiden(adata_female_effective_feature_age_female, resolution=0.2, key_added="leiden_0_3")
df = adata_male_effective_feature_age_male.obs.copy()
x_zuobiao = adata_male_effective_feature_age_male.obsm['X_tsne'][:,0]
y_zuobiao = adata_male_effective_feature_age_male.obsm['X_tsne'][:,1]
df['x'] = x_zuobiao
df['y'] = y_zuobiao
sorted_indices = np.argsort(list(df['dpt_pseudotime']))
ranked_positions = np.argsort(sorted_indices)
sorted_list = sorted(df['true_age'])
df['dpt_pseudotime_map'] = [sorted_list[i] for i in ranked_positions]
# df['cluster'] = df['leiden_0_3']
df['cluster'] = df['kmeans']
centroids = df.groupby('cluster')[['x', 'y']].mean().reset_index()

from scipy.spatial import distance
import numpy as np
import matplotlib as mpl
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.figure(figsize=(6, 5))
sns.scatterplot(x='x', y='y', hue='cluster', data=df, palette='tab10', s=1, edgecolor=None, alpha=1)
offset = 0.05  
for i, centroid in centroids.iterrows():
    cluster_points = df[df['cluster'] == centroid['cluster']]
    centroid_coords = centroid[['x', 'y']].values
    min_dist = np.min([distance.euclidean(centroid_coords, point) for point in cluster_points[['x', 'y']].values])
    adjusted_centroid = centroid_coords.copy()
    while any(distance.euclidean(adjusted_centroid, other_centroid) < offset for other_centroid in centroids[['x', 'y']].values):
        adjusted_centroid += np.random.uniform(-offset, offset, size=2)
    plt.text(adjusted_centroid[0], adjusted_centroid[1], f'Cluster {centroid["cluster"]}', 
             ha='center', va='center', fontsize=12, fontweight='bold', color='black')
# plt.title('Cluster Visualization with Labels', fontsize=1)
plt.xlabel('TSNE-1', fontsize=10)
plt.ylabel('TSNE-2', fontsize=10)
plt.legend(title='Cluster', title_fontsize=10, loc='upper left', fontsize=12, markerscale=5, bbox_to_anchor=(1, 1))
plt.grid(True, alpha=0.3)
plt.tight_layout()
# plt.savefig("./figs-method/Cluster_proage20_feat_male_kmeans.png", dpi=600)
plt.show()

In [ ]:
adata_male_effective_feature_age_male.write('./dzz-harzard/adata_male_effective_feature_age_male.h5ad')

# adata_male_effective_feature_age_male.obs[['eid','true_age','true_agepredmale','dpt_pseudotime_map','kmeans']]
# adata_male_effective_feature_age_male.obs[['eid','true_age','true_agepredmale','dpt_pseudotime_map','kmeans']].to_parquet('./figs-method/male_overage_vs_kmeans_1115.parquet')

In [ ]:
import scanpy as sc
adata_male_effective_feature_age_male.obs['cluster'] = adata_male_effective_feature_age_male.obs['kmeans']
adata_male_effective_feature_age_male_sub = sc.pp.subsample(adata_male_effective_feature_age_male, fraction=0.7, random_state=1, copy=True)
adata_male_effective_feature_age_male_sub.obs.reset_index(drop=True, inplace=True)

In [ ]:
adata_male_effective_feature_age_male_sub.obs[adata_male_effective_feature_age_male_sub.obs['x_zuobiao']<-90]

In [ ]:
adata_male_effective_feature_age_male.obs[['eid','cluster']].to_parquet('./figs-method/male_diff_aging_trajectary_1115.parquet')